# 0. 오늘 수업의 목표
수업이 끝나면 학습자는 아래를 "말로 설명"할 수 있어야 한다.

- groupy가 뭔지: "기준으로 묶고(그룹) → 요약값(집계)을 만든다"
- merge가 뭔지: "두 테이블을 키로 붙여서(조인) 정보 확장한다"
- 문자파열/시간 처리가 왜 필요한지: "분석 가능한 형태로 통일하고 생 피처를 만든다"
- apply/map의 차이: "간단 치환(map) vs 행 단위 규칙(apply)"

# 1) 큰 그림: 원본 로그 → 리포트 테이블

## 1.1) 데이터 분석에서 자주 보는 3단 구조
### 1. 원본 로그 (거래 / 주문 단위)
- 주문 1건, 결제 1건처럼 ‘사건 기록’이 한 행씩 쌓인 데이터
- 행 수가 많고, 값의 형식이 제각각임 (원시 데이터, 데이터가 정리되지 않았다.)
- 시스템이 저장한 그대로라 분석 친화적이지 않음 
<br> <br>
특징
- 행 기준: “무슨 일이 있었나”
- 값이 들쭉날쭉함 (문자열/숫자 섞임, 표기 흔들림)
- 그대로는 리포트(요약표) 만들기 어려움 
<br> <br>
 

### 2. 분석용 테이블 (정제 / 통일 / 파생)
- 분석이 가능하도록 형식과 타입을 통일한 상태
<br> <br>

주로 하는 작업 
- 날짜 형식 통일 → datetime
- 가격/수량 → 숫자
- 결제 여부 → True / False
- 표기 통일 → 공백 제거, 대소문자 통일
- 리포트에 필요한 파생 컬럼 생성 (연_월, 요일, 매출 등) 
<br> <br>

### 3. 요약 테이블 (집계 / 피벗)
GroupBy / 피벗으로 만든 최종 결과표
- 보고서, 대시보드에 바로 붙여 넣을 수 있는 형태 

특징
- 행 수는 적고
- 의미는 압축되어 있음
- "의사결정용 표"

## 1.2) 예시로 보기: 카페 매출 데이터
원본 로그 예시 컬럼
- date, menu, price, qty, paid, store, channel …

이 상태로 바로 집계하면 문제가 발생한다.
- "4,500원" 같은 문자열 때문에 합계/평균 계산이 깨짐
- "Latte", "latte", " Latte "가 서로 다른 메뉴로 집계됨
- 날짜가 문자열이면 월별/요일별 분석이 번거로움

때문에 먼저 분석용 테이블로 바꿀 필요가 있다.
- 정제 / 통일
    - date → datetime
    - price, qty → 숫자
    - paid → True / False
    - menu 표기 통일

- 파생
    - ym (연-월)
    - day_name (요일)
    - sales (매출)

이 단계를 거쳐야 집계 결과가 믿을 수 있게 된다.

# 1.3) 최종 요약표(리포트)가 가져야 할 모습

- 월별 매출 추이
→ 이번 달 vs 지난 달 비교

- 요일별 매출 패턴
→ 어떤 요일이 강한지 (운영 / 인력 배치)

- 메뉴 TOP / 카테고리별 매출
→ 잘 팔리는 메뉴 파악 (메뉴 전략)

- 결제 성공률 / 실패율
→ 오류, 취소, 실패 비중 점검

# 2. GroupBy(그룹화) 이론

## 2.1) GroupBy란?
GroupBy = “기준 컬럼으로 묶고(그룹) → 각 그룹의 요약값(집계)을 만든다” 이다.

원본 로그(주문 1건 = 1행)를 리포트용 요약표로 바꿀 때 가장 많이 쓴다.

- 메뉴별 매출 합계: “Americano가 총 얼마 벌었나?”
- 매장별 주문 수: “광교점은 주문이 몇 건인가?”
- 월별 결제 성공률: “이번 달 결제 실패가 늘었나?”

### groupby 문법구조
```python
데이터프레임.groupby(기준)[집계대상].집계함수()
```
- 원본 데이터프레임을
- groupby(기준): 무엇을 기준으로 묶을지 (문자열 컬럼명 또는 컬럼명 리스트가 온다)
- [집계대상]: 컬럼을 집계할지
- 집계함수(): 어떻게 요약할지(집계 함수) (sum, mean, count, nunique, min, max)


#### 가장 많이 쓰이는 패턴 3개
##### 단일 기준 + 단일 집계
```python
df.groupby("menu")["sales"].sum()
```
- menu 기준으로 묶고
- sales를 합계로 요약
<br> <br>

##### 다중 기준 + 단일 집계
```python
df.groupby(["ym", "menu"])["sales"].sum()
```
- ym, menu 기준으로 묶고
- sales를 합계로 요약한다
<br> <br>

##### 단일/다중 기준 + 여러 집계 (리포트형)
```python
df.groupby("store").agg(
    total_sales=("sales", "sum"),
    orders=("menu", "count"),
    paid_rate=("paid", "mean")
)
```
- store 기준으로 묶고
- 여러 컬럼을 여러 방식으로 요약한다
<br> <br>


##### 결과 형태까지 포함한 문장 구조
```python
데이터프레임
  .groupby(기준)
  .agg(집계정의)
  .reset_index()
```


In [42]:
import pandas as pd

# 주문(원본 로그) 데이터: "주문 1건 = 행 1개"
df = pd.DataFrame([
    {"date": "2026-01-01", "store": "광교점", "menu": "Americano", "sales": 9000, "paid": True},
    {"date": "2026-01-01", "store": "광교점", "menu": "Latte",     "sales": 5000, "paid": True},
    {"date": "2026-01-02", "store": "광교점", "menu": "Latte",     "sales": 0,    "paid": False},
    {"date": "2026-01-03", "store": "수원점", "menu": "Americano", "sales": 4500, "paid": True},
    {"date": "2026-01-03", "store": "수원점", "menu": "Mocha",     "sales": 0,    "paid": False},
])

# 날짜를 datetime으로 바꾸고(dt), 월(ym) 컬럼 생성 (월별 그룹화용)
df["date"] = pd.to_datetime(df["date"])
df["ym"] = df["date"].dt.to_period("M").astype(str)

print("원본 로그")
print(df)

# 메뉴별 매출 합계
menu_sales = df.groupby("menu")["sales"].sum().reset_index()
print("\n메뉴별 매출 합계")
print(menu_sales)

# 매장별 주문 수 (행 개수)
store_orders = df.groupby("store")["menu"].count().reset_index(name="orders")
print("\n매장별 주문 수")
print(store_orders)

# 월별 결제 성공률 (True/False 평균)
monthly_paid_rate = df.groupby("ym")["paid"].mean().reset_index(name="paid_rate")
print("\n월별 결제 성공률")
print(monthly_paid_rate)


원본 로그
        date store       menu  sales   paid       ym
0 2026-01-01   광교점  Americano   9000   True  2026-01
1 2026-01-01   광교점      Latte   5000   True  2026-01
2 2026-01-02   광교점      Latte      0  False  2026-01
3 2026-01-03   수원점  Americano   4500   True  2026-01
4 2026-01-03   수원점      Mocha      0  False  2026-01

메뉴별 매출 합계
        menu  sales
0  Americano  13500
1      Latte   5000
2      Mocha      0

매장별 주문 수
  store  orders
0   광교점       3
1   수원점       2

월별 결제 성공률
        ym  paid_rate
0  2026-01        0.6


## 2.2) GroupBy가 왜 필요할까?
원본 데이터 대부분 "행 단위 사건"이기 때문에 그대로 보면 흐름이 잘 안 보인다.<br>
주문 1건을 봐서는 이번 달 매출이 좋은지 판단할 수 없고, 수백 건을 사람 눈으로 보면 패턴이 안 잡힌다.

때문에 GroupBy로 "요약 단위"를 바꾼다.
- 주문 단위 → 메뉴 단위 / 매장 단위 / 월 단위
- 요약 단위로 바꾸면 비교·추세·순위 같은 리포트가 가능해진다.

In [43]:
import pandas as pd

df = pd.DataFrame([
    {"date": "2026-01-01", "store": "광교점", "menu": "Americano", "sales": 9000},
    {"date": "2026-01-01", "store": "광교점", "menu": "Latte",     "sales": 5000},
    {"date": "2026-01-02", "store": "광교점", "menu": "Latte",     "sales": 5000},
    {"date": "2026-01-03", "store": "수원점", "menu": "Americano", "sales": 4500},
    {"date": "2026-01-03", "store": "수원점", "menu": "Mocha",     "sales": 5500},
])

df["date"] = pd.to_datetime(df["date"])
df["ym"] = df["date"].dt.to_period("M").astype(str)

print("원본 로그(주문 1건 단위):")
print(df)

menu_report = df.groupby("menu")["sales"].sum().reset_index(name="total_sales")
store_report = df.groupby("store")["sales"].sum().reset_index(name="total_sales")
month_report = df.groupby("ym")["sales"].sum().reset_index(name="total_sales")

print("\n메뉴 단위 요약(메뉴별 매출 합계):")
print(menu_report)

print("\n매장 단위 요약(매장별 매출 합계):")
print(store_report)

print("\n월 단위 요약(월별 매출 합계):")
print(month_report)

top_menu = menu_report.sort_values("total_sales", ascending=False)
print("\n메뉴 매출 TOP(정렬로 순위 확인):")
print(top_menu)


원본 로그(주문 1건 단위):
        date store       menu  sales       ym
0 2026-01-01   광교점  Americano   9000  2026-01
1 2026-01-01   광교점      Latte   5000  2026-01
2 2026-01-02   광교점      Latte   5000  2026-01
3 2026-01-03   수원점  Americano   4500  2026-01
4 2026-01-03   수원점      Mocha   5500  2026-01

메뉴 단위 요약(메뉴별 매출 합계):
        menu  total_sales
0  Americano        13500
1      Latte        10000
2      Mocha         5500

매장 단위 요약(매장별 매출 합계):
  store  total_sales
0   광교점        19000
1   수원점        10000

월 단위 요약(월별 매출 합계):
        ym  total_sales
0  2026-01        29000

메뉴 매출 TOP(정렬로 순위 확인):
        menu  total_sales
0  Americano        13500
1      Latte        10000
2      Mocha         5500


## 2.3) 집계 함수(aggregation) 핵심 5종
- sum: 총합 (총매출, 총수량)
- mean: 평균 (평균 매출, 평균 수량, 성공률)
- count: 개수 (주문 건수 = 행 개수)
- nunique: 고유값 개수 (고유 메뉴 수, 고유 고객 수)
- min / max: 최소/최대 (최소/최대 수량, 최대 매출 등)

In [44]:
import pandas as pd

df = pd.DataFrame([
    {"store": "광교점", "menu": "Americano", "qty": 2, "sales": 9000, "paid": True},
    {"store": "광교점", "menu": "Latte",     "qty": 1, "sales": 5000, "paid": True},
    {"store": "광교점", "menu": "Latte",     "qty": 2, "sales": 0,    "paid": False},
    {"store": "수원점", "menu": "Americano", "qty": 1, "sales": 4500, "paid": True},
    {"store": "수원점", "menu": "Mocha",     "qty": 1, "sales": 0,    "paid": False},
])

summary = df.groupby("store").agg(
    total_sales=("sales", "sum"),
    avg_sales=("sales", "mean"),
    orders=("menu", "count"),
    unique_menus=("menu", "nunique"),
    min_qty=("qty", "min"),
    max_qty=("qty", "max"),
    paid_rate=("paid", "mean"),
).reset_index()

print("사용한 집계 함수 출력")
print(summary)


사용한 집계 함수 출력
  store  total_sales    avg_sales  orders  unique_menus  min_qty  max_qty  \
0   광교점        14000  4666.666667       3             2        1        2   
1   수원점         4500  2250.000000       2             2        1        1   

   paid_rate  
0   0.666667  
1   0.500000  


## 2.4) 단일 기준 vs 다중 기준
단일 기준 그룹화는 “기준이 1개”라서 표가 단순하고 직관적이다.
- 예: menu 기준 → 메뉴별 매출 TOP

다중 기준 그룹화는 “기준이 2개 이상”이라서 리포트가 풍부해진다.
- 예: ym(월) + day_name(요일) 기준 → 월별-요일별 매출 패턴

In [83]:
import pandas as pd

df = pd.DataFrame([
    {"date": "2026-01-01", "menu": "Americano", "sales": 9000},
    {"date": "2026-01-02", "menu": "Latte",     "sales": 5000},
    {"date": "2026-01-03", "menu": "Latte",     "sales": 5000},
    {"date": "2026-01-06", "menu": "Americano", "sales": 4500},
    {"date": "2026-01-07", "menu": "Mocha",     "sales": 5500},
])

df["date"] = pd.to_datetime(df["date"])
df["ym"] = df["date"].dt.to_period("M").astype(str)
df["day_name"] = df["date"].dt.day_name()

# 단일기준 그룹화. menu 기준 → 메뉴별 매출 TOP
menu_report = df.groupby("menu")["sales"].sum().reset_index(name="total_sales")

# 다중기준 그룹화. ym(월) + day_name(요일) 기준 → 월별-요일별 매출 패턴
ym_day_report = (
    df.groupby(["ym", "day_name"])["sales"].sum()
      .reset_index(name="total_sales")
)

print("메뉴 단위 요약 ")
print(menu_report)

print("\n 월-요일 단위 요약 (월별·요일별 매출 합계")
print(ym_day_report)


메뉴 단위 요약 
        menu  total_sales
0  Americano        13500
1      Latte        10000
2      Mocha         5500

 월-요일 단위 요약 (월별·요일별 매출 합계
        ym   day_name  total_sales
0  2026-01     Friday         5000
1  2026-01   Saturday         5000
2  2026-01   Thursday         9000
3  2026-01    Tuesday         4500
4  2026-01  Wednesday         5500


## 2.5) agg()가 중요한 이유
실무 리포트는 보통 “한 표에 여러 지표”가 들어간다.<br>
sum 하나만 뽑고 끝내면 리포트가 아니라 중간 결과가 된다.

agg()는 여러 지표를 한 번에 뽑아서 “리포트 표”를 만들게 해준다.

In [46]:
import pandas as pd

# 예시 데이터(주문 1건 = 1행)
df = pd.DataFrame([
    {"store": "광교점", "menu": "Americano", "sales": 9000, "paid": True},
    {"store": "광교점", "menu": "Latte",     "sales": 5000, "paid": True},
    {"store": "광교점", "menu": "Latte",     "sales": 0,    "paid": False},
    {"store": "수원점", "menu": "Americano", "sales": 4500, "paid": True},
    {"store": "수원점", "menu": "Mocha",     "sales": 0,    "paid": False},
])

report = df.groupby("store").agg(
    total_sales=("sales", "sum"),
    orders=("menu", "count"),
    paid_rate=("paid", "mean"),
).reset_index()


print("원본")
print(df)


print("\n매장별 요약")
print(report)

원본
  store       menu  sales   paid
0   광교점  Americano   9000   True
1   광교점      Latte   5000   True
2   광교점      Latte      0  False
3   수원점  Americano   4500   True
4   수원점      Mocha      0  False

매장별 요약
  store  total_sales  orders  paid_rate
0   광교점        14000       3   0.666667
1   수원점         4500       2   0.500000


## 2.6) MultiIndex가 뭐고 왜 불편한가?
다중 기준 groupby를 하면 결과가 MultiIndex(계층 인덱스)로 나오는 경우가 많다. <br>
이 상태는 "표처럼" 다루기가 불편해질 수 있다(merge, 저장, 피벗 등). <br>
그래서 실무에서는 보통 reset_index()로 인덱스를 컬럼으로 내려서 표 형태로 만든다.


In [47]:
import pandas as pd

df = pd.DataFrame([
    {"date": "2026-01-01", "store": "광교점", "sales": 9000},
    {"date": "2026-01-01", "store": "광교점", "sales": 5000},
    {"date": "2026-01-03", "store": "수원점", "sales": 4500},
])

df["date"] = pd.to_datetime(df["date"])
df["ym"] = df["date"].dt.to_period("M").astype(str)

multi = df.groupby(["ym", "store"])["sales"].sum()
print("=== 다중 기준 groupby 결과 (MultiIndex 상태) ===")
print(multi)

flat = multi.reset_index(name="total_sales")
print("\n=== reset_index() 후 (표 형태로 변환) ===")
print(flat)


=== 다중 기준 groupby 결과 (MultiIndex 상태) ===
ym       store
2026-01  광교점      14000
         수원점       4500
Name: sales, dtype: int64

=== reset_index() 후 (표 형태로 변환) ===
        ym store  total_sales
0  2026-01   광교점        14000
1  2026-01   수원점         4500


# 3) 피벗(표 형태 변환) 이론: 리포트용 표 만들기

## 3.1) 피벗이 필요한 순간
피벗은 리포트에서 읽히는 표를 만들기 위한 변환이다.

원본 데이터는 대부분 다음 구조를 가진다.
- 주문 1건 = 1행 구조
- 컬럼: date, menu, store, sales …

이 구조는 데이터는 많지만, 패턴을 한눈에 보기 어렵다


하지만 리포트에서는 보통 이런 표를 원한다.
- 행(Row): 월(예: 2026-01, 2026-02)
- 열(Column): 요일(예: Mon, Tue)
- 값(Value): 매출 합계

이렇게 바꾸면 어느 요일이 강한지, 월별 패턴이 어떻게 바뀌는지가 바로 보인다.

### 코드로 보기
핵심 흐름은 항상 같다.
- GroupBy로 먼저 요약한다 (월 + 요일별 매출 합계)
- 그 결과를 피벗으로 표 모양으로 펼친다

해당 코드를 보면 역할은 명확하다.
- GroupBy는 숫자를 만든다
- Pivot은 표 모양을 만든다

In [48]:
import pandas as pd

# 원본 로그 (주문 1건 = 1행)
df = pd.DataFrame([
    {"date": "2026-01-01", "menu": "Americano", "store": "광교점", "sales": 9000},
    {"date": "2026-01-02", "menu": "Latte",     "store": "광교점", "sales": 5000},
    {"date": "2026-01-03", "menu": "Latte",     "store": "수원점", "sales": 5000},
    {"date": "2026-01-06", "menu": "Americano", "store": "광교점", "sales": 4500},
    {"date": "2026-02-03", "menu": "Mocha",     "store": "수원점", "sales": 5500},
])

# 날짜 파생 컬럼
df["date"] = pd.to_datetime(df["date"])
df["ym"] = df["date"].dt.to_period("M").astype(str)
df["day"] = df["date"].dt.day_name()

print("원본 로그")
print(df[["date", "ym", "day", "menu", "sales"]])

# 1단계: GroupBy로 요약
summary = (
    df.groupby(["ym", "day"])["sales"].sum()
      .reset_index(name="total_sales")
)

print("\nGroupBy 요약(월+요일별 매출)")
print(summary)

# 2단계: Pivot으로 표 형태 변환
report = (
    summary.pivot(index="ym", columns="day", values="total_sales")
    .fillna(0)
)

print("\n피벗 결과(월 x 요일)")
print(report)


원본 로그
        date       ym       day       menu  sales
0 2026-01-01  2026-01  Thursday  Americano   9000
1 2026-01-02  2026-01    Friday      Latte   5000
2 2026-01-03  2026-01  Saturday      Latte   5000
3 2026-01-06  2026-01   Tuesday  Americano   4500
4 2026-02-03  2026-02   Tuesday      Mocha   5500

GroupBy 요약(월+요일별 매출)
        ym       day  total_sales
0  2026-01    Friday         5000
1  2026-01  Saturday         5000
2  2026-01  Thursday         9000
3  2026-01   Tuesday         4500
4  2026-02   Tuesday         5500

피벗 결과(월 x 요일)
day      Friday  Saturday  Thursday  Tuesday
ym                                          
2026-01  5000.0    5000.0    9000.0   4500.0
2026-02     0.0       0.0       0.0   5500.0


## 3.2) 긴 형태(long) → 넓은 형태(wide)
피벗을 이해하는 가장 쉬운 관점은 이거다.
- 긴 형태(long): 요일이 행에 들어 있다
- 넓은 형태(wide): 요일이 열로 펼쳐진다

요약하면 "행에 있던 요일을 열로 펼쳐서 표처럼 만든다"가 피벗이다.

In [49]:
import pandas as pd

long_df = pd.DataFrame([
    {"ym": "2026-01", "day": "Mon", "sales": 120000},
    {"ym": "2026-01", "day": "Tue", "sales":  90000},
    {"ym": "2026-02", "day": "Mon", "sales": 150000},
    {"ym": "2026-02", "day": "Tue", "sales":  80000},
])

# 긴 형태(long)
print("긴 형태(long)")
print(long_df)

# 넓은 형태(wide)
wide_df = long_df.pivot(
    index="ym",
    columns="day",
    values="sales"
).fillna(0)

print("\n넓은 형태(wide)")
print(wide_df)


긴 형태(long)
        ym  day   sales
0  2026-01  Mon  120000
1  2026-01  Tue   90000
2  2026-02  Mon  150000
3  2026-02  Tue   80000

넓은 형태(wide)
day         Mon    Tue
ym                    
2026-01  120000  90000
2026-02  150000  80000


### 긴 형태(long)
- 기준 정보가 행으로 반복된다
- 분석에는 편하지만, 사람이 보기에는 불편하다

### 넓은 형태(wide)
- 기준 중 하나를 열로 펼친다
- 리포트에 바로 쓰기 좋은 형태다

이렇게 기억하자
- long → 분석용 구조
- wide → 리포트용 구조

## 3.3) pivot vs unstack은 뭐가 다른가?
pivot과 unstack은 결과 모양은 비슷하지만 출발점이 다르다.

- 둘 다 결과를 넓은 형태(wide) 로 만든다
- 즉, 행에 있던 기준을 열로 펼친다

사용 가이드
- 표를 “설계”하는 느낌이 필요하면 → pivot
- GroupBy 결과의 인덱스를 바꿔야하면 → unstack

In [50]:
import pandas as pd

# 예시 데이터(일부 조합이 반복되도록 일부러 2행 넣음: 2026-01 + Mon)
df = pd.DataFrame([
    {"ym": "2026-01", "day": "Mon", "sales": 100},
    {"ym": "2026-01", "day": "Mon", "sales": 50},   # 같은 조합이 2개라서 pivot이 바로 안 됨
    {"ym": "2026-01", "day": "Tue", "sales": 80},
    {"ym": "2026-02", "day": "Mon", "sales": 120},
    # 2026-02의 Tue가 없어서 피벗 후 NaN이 생길 수 있음
])

print("원본(long 형태)")
print(df)

원본(long 형태)
        ym  day  sales
0  2026-01  Mon    100
1  2026-01  Mon     50
2  2026-01  Tue     80
3  2026-02  Mon    120


### pivot
지금 내가 가진 데이터에서 행 / 열 / 값이 뭔지 직접 지정해서 표를 만든다

pivot의 전제 조건
- 한 칸(index + column 조합)에 값이 1개여야 한다
- 값이 여러 개면 pivot은 실패한다

사고 흐름
- 같은 조합을 먼저 하나의 값으로 만든다
- 그 다음, 행/열/값을 명시해서 표를 만든다


문장 구조
```python
데이터프레임.pivot(
    index="행 기준",
    columns="열 기준",
    values="값"
)
```
pivot은 GroupBy 이후 이미 요약이 끝난 데이터에서 사용한다

In [51]:
# 1단계: 먼저 요약 (칸당 값 1개 만들기)
summary = (
    df.groupby(["ym", "day"])["sales"]
      .sum()
      .reset_index()
)

print("\nGroupBy 요약 결과")
print(summary)

# 2단계: pivot으로 표 설계
report = summary.pivot(
    index="ym",
    columns="day",
    values="sales"
)

print("\npivot 결과")
print(report)

# 즉, pivot은 표 설계 도구다
# "이걸 행으로, 이걸 열로, 이 값을 채운다"라고 말하는 방식이다


GroupBy 요약 결과
        ym  day  sales
0  2026-01  Mon    150
1  2026-01  Tue     80
2  2026-02  Mon    120

pivot 결과
day        Mon   Tue
ym                  
2026-01  150.0  80.0
2026-02  120.0   NaN


### unstack
이미 GroupBy 결과로 만들어진 계층 인덱스(MultiIndex) 를 열로 펼친다

unstack의 전제 조건
- 대상은 GroupBy 결과
- 보통 결과가 MultiIndex Series다

사고 흐름
- GroupBy를 한다
- 결과가 MultiIndex가 된다
- 인덱스 중 하나를 열로 밀어낸다

문장구조
```python
그룹바이_결과.unstack()

# 또는

그룹바이_결과.unstack(level=인덱스_레벨)
```

In [52]:
# GroupBy 결과 (MultiIndex Series)
multi = df.groupby(["ym", "day"])["sales"].sum()

print("\nGroupBy 결과 (MultiIndex)")
print(multi)

# unstack으로 인덱스 레벨을 열로 펼침
wide = multi.unstack()

print("\nunstack 결과")
print(wide)
# 즉, unstack은 이미 만들어진 구조를 펼치는 도구다


GroupBy 결과 (MultiIndex)
ym       day
2026-01  Mon    150
         Tue     80
2026-02  Mon    120
Name: sales, dtype: int64

unstack 결과
day        Mon   Tue
ym                  
2026-01  150.0  80.0
2026-02  120.0   NaN


### 같은 결과, 다른 출발점
#### pivot
- 출발점: 일반 DataFrame
- 목적: 리포트 표 설계
- 질문 형태:
    - 행은 뭐냐?
    - 열은 뭐냐?
    - 값은 뭐냐?

→ 사람 중심, 리포트 중심

#### unstack
- 출발점: GroupBy 결과(MultiIndex)
- 목적: 인덱스 구조 변경
- 질문 형태:
    - 이 인덱스 레벨을 열로 바꿔라

→ 판다스 내부 구조 처리용



In [53]:
# pivot 방식
summary = df.groupby(["ym", "day"])["sales"].sum().reset_index()
pivot_table = summary.pivot(index="ym", columns="day", values="sales")

# unstack 방식
multi = df.groupby(["ym", "day"])["sales"].sum()
unstack_table = multi.unstack()


차이점은
- pivot: DataFrame → 표
- unstack: MultiIndex → 표

뿐이다.

## 3.4) 피벗에서 자주 만나는 실수 포인트 2개
피벗은 문법보다 전제 조건을 모르면 계속 막힌다.

아래 2가지만 이해하면 피벗에서 대부분의 에러를 피할수 있다.

### 값이 여러 개라서 피벗이 안 되는 경우
pivot은 “한 칸 = 값 1개”를 전제로 한다.
- (ym, day) 조합 하나당
- 값(sales)은 딱 하나여야 한다.


왜 문제가 발생하나?
- 주문 1건 = 1행
- 같은 월 + 같은 요일 주문이 여러 건 존재

In [54]:
import pandas as pd

df = pd.DataFrame([
    {"ym": "2026-01", "day": "Mon", "sales": 100},
    {"ym": "2026-01", "day": "Mon", "sales": 50},   # 같은 조합이 반복
    {"ym": "2026-01", "day": "Tue", "sales": 80},
    {"ym": "2026-02", "day": "Mon", "sales": 120},
])

# 이 상태에서 바로 pivot을 하면,
# 2026-01 + Mon 칸에 100과 50 두 개가 동시에 들어가려 해 충돌 발생

In [55]:
# 1단계: GroupBy로 먼저 요약한다
# (칸당 값 1개 만들기)
summary = (
    df.groupby(["ym", "day"])["sales"]
      .sum()
      .reset_index(name="total_sales")
)

print("GroupBy 요약")
print(summary)


GroupBy 요약
        ym  day  total_sales
0  2026-01  Mon          150
1  2026-01  Tue           80
2  2026-02  Mon          120


In [56]:
# 2단계: 그 다음에 pivot을 한다
report = summary.pivot(
    index="ym",
    columns="day",
    values="total_sales"
)

print("\npivot 결과")
print(report)



pivot 결과
day        Mon   Tue
ym                  
2026-01  150.0  80.0
2026-02  120.0   NaN


### 빈 칸(NaN)이 생기는 것
NaN은 에러가 아니라 "그 조합의 데이터가 없다"는 뜻이다.

왜 NaN이 생기나?
- 행: 월
- 열: 요일

그런데 현실 데이터에서는, 어떤 월에는 화요일 매출이 아예 없을 수 있다.

In [57]:
summary = pd.DataFrame([
    {"ym": "2026-01", "day": "Mon", "total_sales": 150},
    {"ym": "2026-01", "day": "Tue", "total_sales": 80},
    {"ym": "2026-02", "day": "Mon", "total_sales": 120},
    # 2026-02의 Tue는 없음
])
summary

,ym,day,total_sales
0,2026-01,Mon,150
1,2026-01,Tue,80
2,2026-02,Mon,120


In [58]:
report = summary.pivot(
    index="ym",
    columns="day",
    values="total_sales"
)

print("피벗 결과 (NaN 발생)")
print(report)


피벗 결과 (NaN 발생)
day        Mon   Tue
ym                  
2026-01  150.0  80.0
2026-02  120.0   NaN


In [59]:
report_filled = report.fillna(0)

print("\n리포트용 (0으로 채움)")
print(report_filled)

# 결측 자체가 의미 있는 경우
# 측정 실패 나 데이터 누락 분석인 경우 NaN을 유지한다.


리포트용 (0으로 채움)
day        Mon   Tue
ym                  
2026-01  150.0  80.0
2026-02  120.0   0.0


# 4. 문자열 처리 이론(실무형) 확장

## 4.1) 왜 문자열 처리가 필수인가?
실무 데이터에서 문자열은 거의 항상 정리되지 않은 상태로 들어온다.

문제는 이게 단순히 보기만 지저분한 수준이 아니라는 점이다.

- 분석 결과가 틀리게 나오거나
- 아예 계산 자체가 불가능해진다.

즉, 문자열 처리는 선택이 아닌 전처리의 필수 단계다.

### 발생하는 문제 1. 같은 값인데 다르게 저장됨 (표기 흔들림)
실무 데이터에서는 같은 의미의 값이 여러 형태로 저장되는 경우가 매우 많다.
- " Latte "
- "latte"
- "LATTE"

사람 눈에는 전부 같은 메뉴로 보인다. 하지만 컴퓨터 입장에서는 완전히 다른 값이다.

떄문에 이 상태에서 GroupBy를 하면
- Latte가 하나의 그룹으로 묶이지 않는다
- 여러 그룹으로 쪼개진다
- 결과적으로 메뉴별 집계가 깨진다

이 때문에 데이터는 맞게 들어와 있어도 문자열 정리를 안 했다는 이유만으로 분석 결과가 틀어진다.


### 발생하는 문제 2. 숫자가 문자열로 섞여 들어옴
실무 데이터에서는 숫자처럼 보이지만 실제로는 문자열인 값도 매우 흔하다.
- "4,500원"
- "5000"
- "5,000원"

눈으로 보면 전부 가격나  하지만 컴퓨터는 이렇게 인식한다.
- 쉼표가 들어가 있음
- 통화 기호가 붙어 있음

이렇게 숫자가 아닌 문자열로 인식하기 때문에 계산이나 집께를 할 수가 없다.

### 발생하는 문제 정리
문자열 전처리를 하지 않으면 리포트에서 다음과 같은 문제가 발생한다.

- 메뉴별 매출 TOP이 잘못 나온다 (같은 메뉴가 여러 행으로 분리됨)

- 가격 합계나 평균이 맞지 않는다 (문자열이라 계산이 안 되거나 일부만 변환됨)

- 필터 조건이 제대로 걸리지 않는다 (공백, 대소문자 차이 때문에 조건 누락)

이런 문제들은 코드가 틀려서가 아닌 데이터 상태가 분석 불가능했기 때문이다.


## 4.2) 초보자 기준 핵심 5가지 + 예시코드
문자열 처리는 꾸미기 작업이 아니다.
- 같은 의미의 값은 같게 만들고
- 계산할 값은 숫자로 바꾸는 작업이다

즉, 문자열 처리의 목적은 분석 전에 데이터를 분석 가능한 상태로 만드는 것이다.


코드를 보며 직접 문제를 해결해보자

In [60]:
import pandas as pd

df = pd.DataFrame([
    {"menu": " Latte ",        "price": "4,500원", "note": "NEW member"},
    {"menu": "latte",          "price": "5000",     "note": "vip MEMBER"},
    {"menu": "LATTE",          "price": "5,000원",  "note": "Member coupon"},
    {"menu": "Americano",      "price": "4500원",   "note": "guest"},
    {"menu": "Vanilla Latte",  "price": "5,800원",  "note": "new_member"},
])

print("원본 데이터")
print(df)


원본 데이터
            menu   price           note
0         Latte   4,500원     NEW member
1          latte    5000     vip MEMBER
2          LATTE  5,000원  Member coupon
3      Americano   4500원          guest
4  Vanilla Latte  5,800원     new_member


### 공백 제거
공백이 사라지면 같은 값인지 비교할 수 있는 상태가 된다.
```python
공백을_삭제할_값.strip()
```

In [61]:
df["menu_clean"] = df["menu"].str.strip()

print("\nstrip 적용 결과")
print(df[["menu", "menu_clean"]])



strip 적용 결과
            menu     menu_clean
0         Latte           Latte
1          latte          latte
2          LATTE          LATTE
3      Americano      Americano
4  Vanilla Latte  Vanilla Latte


### 대소문자 통일: lower / title

문자열 비교에서 대소문자 차이는 다른 값이다.
그래서 기준을 하나로 정해야 한다.

- title(): 출력용, 리포트에서 보기 좋게 만들 때 사용

```python
바꿀값.title()
```

In [62]:
df["menu_lower"] = df["menu_clean"].str.lower()
df["menu_title"] = df["menu_clean"].str.title()

print("\nlower / title 적용 결과")
print(df[["menu_clean", "menu_lower", "menu_title"]])



lower / title 적용 결과
      menu_clean     menu_lower     menu_title
0          Latte          latte          Latte
1          latte          latte          Latte
2          LATTE          latte          Latte
3      Americano      americano      Americano
4  Vanilla Latte  vanilla latte  Vanilla Latte


### 포함 여부 필터: contains

문자열 컬럼에서 특정 단어가 들어간 행만 뽑고 싶을 때 사용한다.

- case=False: 대소문자 무시
- na=False: 결측값 때문에 에러 나는 것 방지

```python
.contains()
```

In [63]:
member_df = df[df["note"].str.contains("member", case=False, na=False)]

print("\ncontains로 member 관련 행만 필터")
print(member_df[["note"]])



contains로 member 관련 행만 필터
            note
0     NEW member
1     vip MEMBER
2  Member coupon
4     new_member


### 치환: replace (문자열 → 숫자 전환의 핵심)
가격 데이터는 대부분 문자열로 들어온다. 쉼표, 단위, 기호가 붙어 있기 때문이다.

때문에 다음 과정을 거쳐야 한다.
1. 문자열에서 불필요한 문자 제거
2. 숫자로 변환

In [64]:
df["price_num"] = (
    df["price"]
      .str.replace(",", "", regex=False)
      .str.replace("원", "", regex=False)
)

df["price_num"] = pd.to_numeric(df["price_num"], errors="coerce")

print("\nreplace + 숫자 변환 결과")
print(df[["price", "price_num"]])



replace + 숫자 변환 결과
    price  price_num
0  4,500원       4500
1    5000       5000
2  5,000원       5000
3   4500원       4500
4  5,800원       5800


### 분리: split
문자열 하나에 정보가 여러 개 섞여 있을 때 사용한다.

In [65]:
df["menu_first_word"] = df["menu_title"].str.split().str[0]

print("\nsplit 적용 결과 (첫 단어 추출)")
print(df[["menu_title", "menu_first_word"]])



split 적용 결과 (첫 단어 추출)
      menu_title menu_first_word
0          Latte           Latte
1          Latte           Latte
2          Latte           Latte
3      Americano       Americano
4  Vanilla Latte         Vanilla


# 5. 시간 데이터 처리 이론(실무형)

## 5.1) 왜 날짜 처리가 중요한가?
- 월별 매출
- 주간 매출
- 요일별 패턴
- 시간대별 피크 분석

이 분석들의 공통점은  전부 시간 데이터를 기준으로 집계한다는 점이다.

pandas는 시간 데이터 처리하기 위한 전제조건은 날짜가 문자열(str)이 아니라 datetime 타입이어야 하는 점이다.

In [66]:
import pandas as pd

df = pd.DataFrame({
    "date_str": ["2026/01/02", "2026/01/10", "2026/01/03"],
    "sales": [100, 200, 150]
})

df

,date_str,sales
0,2026/01/02,100
1,2026/01/10,200
2,2026/01/03,150


### 문제 1. 문자열 날짜는 정렬이 깨진다

In [67]:
print("문자열 기준 정렬")
print(df.sort_values("date_str"))  # 문자열 기준 정렬

print("\ndatetime 기준 정렬")
df["date"] = pd.to_datetime(df["date_str"])
print(df.sort_values("date"))      # datetime 기준 정렬

문자열 기준 정렬
     date_str  sales
0  2026/01/02    100
2  2026/01/03    150
1  2026/01/10    200

datetime 기준 정렬
     date_str  sales       date
0  2026/01/02    100 2026-01-02
2  2026/01/03    150 2026-01-03
1  2026/01/10    200 2026-01-10


이런식으로 문자열 상태에서는 01/10이 01/03보다 먼저 올 수도 있다

datetime으로 바꾸면 실제 시간 순서대로 정확히 정렬된다.

특히나 dd/mm/yyyy 같은 형식은 문자열 정렬과 시간 순서가 더 자주 어긋난다.

### 문제 2. 파생 피처를 만들 수 없다
문자열 날짜 상태에서는 시간 분석의 절반 이상을 할 수 없다.

대표적인 파생 피처는 다음과 같다.
- 연도(year)
- 월(month)
- 요일(day_name)
- 주차(week)
- 연-월(ym)

이때 만약 날짜가 문자열이면
- pandas의 .dt 기능을 쓸 수 없다
- month, day_name 같은 파생 컬럼을 만들 수 없다
- 월별/요일별 분석이 막힌다

### 문제 3. 기간 필터링이 어렵다
datetime으로 바꾸지 않으면 기간 필터링은 항상 불안정하다.

실무에서는 이런 요구가 매우 흔하다.
- "2026년 1월만 보기"
- "특정 기간만 잘라서 분석하기"
- "이번 달 vs 지난 달 비교"

이 작업들은 전부 날짜 계산 기반이다.

문자열 상태에서는
- 포함/제외 경계값 실수가 늘어나고 조건이 복잡해지고 결과를 신뢰하기 어려워진다

## 5.2) 핵심 개념: "datetime으로 바꾸고 파생 피처 만든다"
초보자 기준으로 시간 데이터 처리의 핵심은 이 한 줄로 요약된다.

- 문자열 날짜 → pd.to_datetime()으로 변환
- .dt로 파생 피처 생성
- 리포트용으로 연-월(ym) 같은 기준 컬럼을 만든다 <br> <br>

문장 구조
```python
datetime_타입_Series.dt.속성

#또는

datetime_타입_Series.dt.메소드()
``` 
- 주의사항
    - 반드시 Series여야 한다
- 속성엔 다음과 같은 값이 들어올수 있다.
    - year
    - month
    - day
    - hour 등...
- 메소드엔 다음과 같은 키워드가 들어갈수 있다.
    - .to_period("M") → 월 단위 기간으로 변환 
    - .floor("D") → 날짜를 하루 단위로 내림 (시간 제거, 00:00:00으로 맞춤)
    - .strftime("%Y-%m") → 날짜를 "2026-01" 같은 문자열 형식으로 변환



In [68]:
import pandas as pd

df = pd.DataFrame({
    "date": ["2026-01-01", "2026-01-03", "2026-02-01"],
    "sales": [10000, 15000, 20000]
})

In [69]:
# 1) datetime 변환
df["date"] = pd.to_datetime(df["date"])

# 2) 파생 피처 생성
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day_name"] = df["date"].dt.day_name()

# 3) 리포트용 연-월 키
df["ym"] = df["date"].dt.to_period("M").astype(str)

print(df)

        date  sales  year  month  day_name       ym
0 2026-01-01  10000  2026      1  Thursday  2026-01
1 2026-01-03  15000  2026      1  Saturday  2026-01
2 2026-02-01  20000  2026      2    Sunday  2026-02


# 6) 데이터의 결합 (merge / join / concat)

###### merge와 join은 같은 개념

## 6.1) 결합은 왜 필요한가?
실무에서 사용하는 원본 로그에는 분석에 필요한 정보가 한 테이블에 다 들어있는 경우가 거의 없다.

때문에 분석하기 전에 다른 테이블의 정보를 옆으로 붙여서 분석 가능한 형태로 확장한다.


예시: 카페 데이터
- 주문 로그 테이블
    - date, menu, sales, customer_id 는 있음
- 메뉴 정보 테이블
    - menu → category(커피 / 디저트 등)
- 고객 정보 테이블
    - customer_id → city, grade

주문 데이터만으로는
- 카테고리별 매출
- 지역별 매출
- VIP 고객 매출

이런식의 분석이 불가능하다.

In [70]:
import pandas as pd

orders = pd.DataFrame([
    {"order_id": 1, "date": "2026-01-01", "menu": "Latte",     "sales": 5000, "customer_id": "C01"},
    {"order_id": 2, "date": "2026-01-01", "menu": "Americano", "sales": 4500, "customer_id": "C02"},
    {"order_id": 3, "date": "2026-01-02", "menu": "Cake",      "sales": 6000, "customer_id": "C03"},
])

menu_map = pd.DataFrame([
    {"menu": "Latte",     "category": "Coffee"},
    {"menu": "Americano", "category": "Coffee"},
    # Cake는 매핑 누락 상태
])

customers = pd.DataFrame([
    {"customer_id": "C01", "city": "Suwon",  "grade": "VIP"},
    {"customer_id": "C02", "city": "Yongin", "grade": "NEW"},
    {"customer_id": "C03", "city": "Suwon",  "grade": "NORMAL"},
])
print("주문 로그 데이터")
print(orders)

print("\n 메뉴 정보 데이터")
print(menu_map)

print("\n 고객 정보 데이터")
print(customers)


주문 로그 데이터
   order_id        date       menu  sales customer_id
0         1  2026-01-01      Latte   5000         C01
1         2  2026-01-01  Americano   4500         C02
2         3  2026-01-02       Cake   6000         C03

 메뉴 정보 데이터
        menu category
0      Latte   Coffee
1  Americano   Coffee

 고객 정보 데이터
  customer_id    city   grade
0         C01   Suwon     VIP
1         C02  Yongin     NEW
2         C03   Suwon  NORMAL


## 6.2) merge = "키로 붙이는 조인"
merge는 pandas에서 가장 많이 쓰는 결합 방식이다.
- 공통 키(key)를 기준으로 두 테이블을 옆으로 붙인다

```python
왼쪽_데이터프레임.merge(
    오른쪽_데이터프레임,
    on="공통_키",
    how="결합_방식")
```
- 기준은 항상 왼쪽 데이터프레임
- 오른쪽에서 정보를 가져와 붙인다


In [71]:
merged = orders.merge(
    menu_map,
    on="menu",
    how="left"
)

print(merged)


   order_id        date       menu  sales customer_id category
0         1  2026-01-01      Latte   5000         C01   Coffee
1         2  2026-01-01  Americano   4500         C02   Coffee
2         3  2026-01-02       Cake   6000         C03      NaN


orders를 기준으로
- menu를 키로
- menu_map의 category를 붙인다

merge의 결과로
- orders에는 없던 category 컬럼이 생긴다
- Cake처럼 매핑이 없는 값은 결측값이 된다

### 결합 이후 분석이 달라진다
결합 전엔 메뉴별 매출까지만 가능했으나, <br>
결합 후엔 카테고리별 매출로 확장 가능해 졌다.

In [72]:
cat_sales = merged.groupby("category")["sales"].sum()

print(cat_sales)


category
Coffee    9500
Name: sales, dtype: int64


## 6.3) how 옵션 (초보자 필수 3종)
how는 결합 후에 어떤 행(row)을 남길지 결정하는 옵션이다.

how = “기준을 어디에 두고 행을 남길 것인가”

### left — 왼쪽(원본) 유지 [LEFT JOIN]

```python
왼쪽.merge(오른쪽, on=키, how="left")

```
- 왼쪽 데이터프레임의 행은 전부 유지
- 오른쪽에 매핑이 없으면 NaN

언제 사용하나?
- 실무에서 가장 많이 쓰는 옵션
- 원본 로그는 보존하고
- 매핑 누락을 NaN으로 확인할 때

In [ ]:
left_join = orders.merge(menu_map, on="menu", how="left")

print("주문 로그 데이터")
print(orders)

print("\n 메뉴 정보 데이터")
print(menu_map)

print("\n 주문 로그+ 메뉴 정보 데이터 (LEFT)")
print(left_join)

주문 로그 데이터
   order_id        date       menu  sales customer_id
0         1  2026-01-01      Latte   5000         C01
1         2  2026-01-01  Americano   4500         C02
2         3  2026-01-02       Cake   6000         C03

 메뉴 정보 데이터
        menu category
0      Latte   Coffee
1  Americano   Coffee

 주문 로그+ 메뉴 정보 데이터 (LEFT)
   order_id        date       menu  sales customer_id category
0         1  2026-01-01      Latte   5000         C01   Coffee
1         2  2026-01-01  Americano   4500         C02   Coffee
2         3  2026-01-02       Cake   6000         C03      NaN


### inner — 양쪽에 모두 있는 것만 [INNER JOIN]
```python
왼쪽.merge(오른쪽, on=키, how="inner")
```
- 양쪽 테이블에 공통으로 존재하는 키만 남김
- 매핑 안 되는 행은 아예 제거됨

언제 사용하나?
- 매핑이 제대로 됐는지
- 정합성 확인할 때

In [74]:
inner_join = orders.merge(menu_map, on="menu", how="inner")

print("주문 로그 데이터")
print(orders)

print("\n 메뉴 정보 데이터")
print(menu_map)

print("\n 주문 로그+ 메뉴 정보 데이터 (INNER)")
print(inner_join)


주문 로그 데이터
   order_id        date       menu  sales customer_id
0         1  2026-01-01      Latte   5000         C01
1         2  2026-01-01  Americano   4500         C02
2         3  2026-01-02       Cake   6000         C03

 메뉴 정보 데이터
        menu category
0      Latte   Coffee
1  Americano   Coffee

 주문 로그+ 메뉴 정보 데이터 (INNER)
   order_id        date       menu  sales customer_id category
0         1  2026-01-01      Latte   5000         C01   Coffee
1         2  2026-01-01  Americano   4500         C02   Coffee


### outer — 전체 합치기
```python
왼쪽.merge(오른쪽, on=키, how="outer")
```
- 양쪽 테이블의 모든 행을 전부 유지
- 한쪽에만 있으면 NaN

언제 사용하나?
- 어느 쪽에만 있는 값이 있는지
- 누락 / 신규 항목 탐색할 때

In [75]:
outer_join = orders.merge(menu_map, on="menu", how="outer")

print("주문 로그 데이터")
print(orders)

print("\n 메뉴 정보 데이터")
print(menu_map)

print("\n 주문 로그+ 메뉴 정보 데이터 (outer)")
print(outer_join)


주문 로그 데이터
   order_id        date       menu  sales customer_id
0         1  2026-01-01      Latte   5000         C01
1         2  2026-01-01  Americano   4500         C02
2         3  2026-01-02       Cake   6000         C03

 메뉴 정보 데이터
        menu category
0      Latte   Coffee
1  Americano   Coffee

 주문 로그+ 메뉴 정보 데이터 (outer)
   order_id        date       menu  sales customer_id category
0         2  2026-01-01  Americano   4500         C02   Coffee
1         3  2026-01-02       Cake   6000         C03      NaN
2         1  2026-01-01      Latte   5000         C01   Coffee


### how 옵션 한눈에 보기

left
- 왼쪽(원본) 데이터의 행을 전부 유지,<br>매핑이 없으면 NaN으로 표시
- 원본 로그 유지 + 매핑 누락 확인할 때 사용

<br>

inner
- 양쪽 테이블에 모두 있는 키만 남김,<br>매핑 안 된 행은 제거됨
- 매핑 정합성 확인용으로 사용

<br>

outer
- 양쪽 테이블의 모든 행을 전부 유지,<br>한쪽에만 있으면 NaN 발생
- 누락 데이터나 신규 항목 탐색할 때 사용

In [76]:
left_join  = orders.merge(menu_map, on="menu", how="left")
inner_join = orders.merge(menu_map, on="menu", how="inner")
outer_join = orders.merge(menu_map, on="menu", how="outer")

print("\n  (left)")
print("left\n", left_join)

print("\n  (inner)")
print("inner\n", inner_join)

print("\n  (outer)")
print("outer\n", outer_join)


  (left)
left
    order_id        date       menu  sales customer_id category
0         1  2026-01-01      Latte   5000         C01   Coffee
1         2  2026-01-01  Americano   4500         C02   Coffee
2         3  2026-01-02       Cake   6000         C03      NaN

  (inner)
inner
    order_id        date       menu  sales customer_id category
0         1  2026-01-01      Latte   5000         C01   Coffee
1         2  2026-01-01  Americano   4500         C02   Coffee

  (outer)
outer
    order_id        date       menu  sales customer_id category
0         2  2026-01-01  Americano   4500         C02   Coffee
1         3  2026-01-02       Cake   6000         C03      NaN
2         1  2026-01-01      Latte   5000         C01   Coffee


### 실무 팁: 매핑 누락 찾기 (indicator)
매핑표를 붙일 때의 정석 흐름은 이렇다.
- left로 붙인다
- NaN이 생긴 행을 확인한다
- 매핑표를 고친다
-

In [77]:
check = orders.merge(
    menu_map,
    on="menu",
    how="left",
    indicator=True
)

missing = check[check["_merge"] == "left_only"]
print(missing[["menu", "order_id"]])


   menu  order_id
2  Cake         3


indicator의 의미
- both → 양쪽에 있음
- left_only → 왼쪽에만 있음 (매핑 누락)
- right_only → 오른쪽에만 있음

# 7. apply vs map 이론
- map → 한 컬럼 값들을 딕셔너리로 치환할 때

- apply → 규칙이 필요하거나 여러 컬럼을 같이 판단해야 할 때

## 7.1) map — “값 치환 / 매핑”에 특화
map은 한 컬럼의 값들을 1:1로 바꿀 때 가장 적합하다.<br>
즉, "이 값이면 저 값으로 바꿔라"가 명확헌 경우 map을 쓴다.

<br>

``` python
Series.map(매핑_딕셔너리)
```
- 대상은 항상 단일 컬럼(Series)
- 기준은 딕셔너리의 key → value

어디에 사용하나?
- 코드 → 이름 치환
- 숫자 코드 → 등급명
- 간단한 메뉴 / 카테고리 매핑

In [78]:
import pandas as pd

df = pd.DataFrame({
    "channel": ["kiosk", "app", "kiosk", "web"]
})

channel_map = {
    "kiosk": "키오스크",
    "app": "앱",
    "web": "웹"
}
print("매핑 전")
print(df)

# channel 컬럼의 값들을 channel_map 기준으로 그대로 치환한다
df["channel_name"] = df["channel"].map(channel_map)
print("\n  매핑 후")
print(df)

매핑 전
  channel
0   kiosk
1     app
2   kiosk
3     web

  매핑 후
  channel channel_name
0   kiosk         키오스크
1     app            앱
2   kiosk         키오스크
3     web            웹


### 자주하는 실수포인트
- 매핑표에 없는 값은 NaN이 된다
- 그래서 항상 누락을 한 번 확인하는 게 좋다

In [79]:
missing = df[df["channel_name"].isna()]
print(missing)


Empty DataFrame
Columns: [channel, channel_name]
Index: []


## 7.2) apply — “규칙(로직)”이 필요할 때
apply는 딕셔너리로 표현할 수 없는 규칙이 있을 때 쓴다.

- 조건문이 필요하거나
- 계산 로직이 있거나
- 여러 컬럼을 동시에 봐야 할때

apply가 필요하다.




### 문장 구조 (단일 컬럼)
```python
Series.apply(함수)
```
- 한 값씩 함수에 넣어서 처리
- 단일 컬럼 기준 로직

In [80]:
import pandas as pd

df = pd.DataFrame({"sales": [0, 3000, 12000]})

def grade_sales(x):
    if x == 0:
        return "C"
    elif x < 10000:
        return "B"
    else:
        return "A"

df["grade"] = df["sales"].apply(grade_sales)
print(df)


   sales grade
0      0     C
1   3000     B
2  12000     A


### 문장 구조 (행 단위)
```python
DataFrame.apply(함수, axis=1)
```
- 여러 컬럼을 동시에 보고 판단
- axis=1은 “행 기준”이라는 뜻


In [81]:
import pandas as pd

df = pd.DataFrame([
    {"sales": 12000, "paid": True},
    {"sales": 5000,  "paid": True},
    {"sales": 8000,  "paid": False},
])

def final_label(row):
    if row["paid"] is False:
        return "FAIL"
    if row["sales"] >= 10000:
        return "A"
    return "B"

df["label"] = df.apply(final_label, axis=1)
print(df)

   sales   paid label
0  12000   True     A
1   5000   True     B
2   8000  False  FAIL


## 7.3) apply 사용시 주의사항
apply는 보통 파이썬 함수가 행마다 실행되기 때문에 느리다.<br>
그래서 데이터가 커지면 속도가 급격히 느려질 수 있다.<br>
같은 로직이라도 조건식으로 한 번에 처리하면 훨씬 빠르다.

In [ ]:
import pandas as pd

df = pd.DataFrame({"sales": [0, 3000, 12000]})

df["grade_vec"] = "B"
df.loc[df["sales"] == 0, "grade_vec"] = "C"
df.loc[df["sales"] >= 10000, "grade_vec"] = "A"

print(df)


   sales grade_vec
0      0         C
1   3000         B
2  12000         A


# 8. 오늘 수업 정리

"원본 로그를 문자열/날짜 정리로 계산 가능하게 만들고, GroupBy로 요약하고, Merge로 정보 확장해서, 리포트용 표(요약테이블)를 만든다."

# 9. 초보자용 퀴즈(이론 체크, 5문항)
1. GroupBy는 왜 필요한가? "원본 로그"와 비교해서 설명해보세요.
2. paid_rate = mean(paid)가 "비율"이 되는 이유는?
3. MultiIndex가 불편한 이유와 해결 방법 1가지는?
4. merge의 left와 inner의 차이를 “매핑 누락 찾기” 관점에서 설명해보세요.
5. map과 apply는 언제 각각 쓰는가? 예시 1개씩 들어보세요.

## 정답
1. 원본 로그는 거래 단위 데이터기 때문에 그대로는 요약이 안 된다.<br>GroupBy는 같은 기준(메뉴, 날짜 등)으로 묶어서 합계·평균 같은 리포트용 집계를 만들기 위해 필요하다.

2. paid가 True/False일 때, True=1, False=0으로 계산된다.<br>그래서 평균은 전체 중 True의 비율이 된다.

3. 컬럼/인덱스 접근이 복잡해지고 후처리가 귀찮아진다.<br>reset_index()로 평평한 형태로 바꾸는 게 가장 간단한 해결책이다.

4. left는 원본을 전부 남겨서 매핑 누락이 NaN으로 드러난다.<br>inner는 매핑 안 된 행이 사라져서 누락을 찾기 어렵다.

5. map은 값 치환처럼 1:1 매핑일 때 쓴다.<br>apply는 조건이나 규칙이 필요할 때 쓴다